In [1]:
import pandas as pd
import numpy as np

In [2]:
%cd /Users/alex/PETsARD

/Users/alex/PETsARD


In [3]:
import PETsARD

In [4]:
loader = PETsARD.Loader(filepath = '[NHANES] B.csv',
                                header_exist = False,
                                header_names = ['gen','age','race','edu','mar','bmi','dep','pir','gh','mets','qm','dia']
                               )
print(loader.data.head(1))

    gen   age   race       edu      mar        bmi  dep  pir   gh  mets  qm  \
0  Male  62.0  White  Graduate  Married  27.799999    0    0  0.0   0.0  Q2   

   dia  
0    1  


In [5]:
df = loader.data

In [6]:
metadata = {'metadata_col': {
                'gen': {'type': 'categorical', 'na_percentage': 0.0},
                'age': {'type': 'numerical', 'na_percentage': 0.0},
                'race': {'type': 'categorical', 'na_percentage': 0.0},
                'edu': {'type': 'categorical', 'na_percentage': 0.0},
                'mar': {'type': 'categorical', 'na_percentage': 0.0},
                'bmi': {'type': 'numerical', 'na_percentage': 0.0},
                'dep': {'type': 'categorical', 'na_percentage': 0.0},
                'pir': {'type': 'categorical', 'na_percentage': 0.0},
                'gh': {'type': 'numerical', 'na_percentage': 0.0},
                'mets': {'type': 'numerical', 'na_percentage': 0.0},
                'qm': {'type': 'categorical', 'na_percentage': 0.0},
                'dia': {'type': 'categorical', 'na_percentage': 0.0}
                },
             'metadata_global':{
                 'row_num': 4190,
                 'col_num': 12,
                 'na_percentage': 0.0
                }
            }

In [7]:
pm = PETsARD.HyperProcessor(metadata)

In [8]:
from PETsARD.Processor.Missingist import *
from PETsARD.Processor.Outlierist import *
from PETsARD.Processor.Encoder import *
from PETsARD.Processor.Scaler import *

In [9]:
df.loc[[1,2,3,4,5], 'gen'] = np.nan
df.loc[[10,20,30,40,5], 'age'] = np.nan

In [18]:
df.loc[[10,20,30,40,5], 'age']

10   NaN
20   NaN
30   NaN
40   NaN
5    NaN
Name: age, dtype: float32

In [40]:
# test global outlierist transformation

config_1 = {'missingist': {'gen': Missingist_Drop(), 'age': Missingist_Mean()},
            'outlierist': {'gen': Outlierist_LOF(), 'age': None},
            'encoder': {},
            'scaler': {}}

In [41]:
pm.set_config(config_1)

In [42]:
pm.get_config()

{'missingist': {'gen': None,
  'age': <PETsARD.Processor.Missingist.Missingist_Simple at 0x15287e4d0>,
  'race': None,
  'edu': None,
  'mar': None,
  'bmi': None,
  'dep': None,
  'pir': None,
  'gh': None,
  'mets': None,
  'qm': None,
  'dia': None},
 'outlierist': {'gen': <PETsARD.Processor.Outlierist.Outlierist_LOF at 0x15287e5c0>,
  'age': None,
  'race': None,
  'edu': None,
  'mar': None,
  'bmi': None,
  'dep': None,
  'pir': None,
  'gh': None,
  'mets': None,
  'qm': None,
  'dia': None},
 'encoder': {'gen': None,
  'age': None,
  'race': None,
  'edu': None,
  'mar': None,
  'bmi': None,
  'dep': None,
  'pir': None,
  'gh': None,
  'mets': None,
  'qm': None,
  'dia': None},
 'scaler': {'gen': None,
  'age': None,
  'race': None,
  'edu': None,
  'mar': None,
  'bmi': None,
  'dep': None,
  'pir': None,
  'gh': None,
  'mets': None,
  'qm': None,
  'dia': None}}

In [43]:
pm.fit(df, ['missingist'])

In [44]:
df_transformed = pm.transform(df)

In [45]:
df_transformed

,gen,age,race,edu,mar,bmi,dep,pir,gh,mets,qm,dia
0,Male,62.0,White,Graduate,Married,27.799999,0,0,0.0,0.0,Q2,1
1,NaN,53.0,White,HighSchool,Divorced,30.799999,0,1,0.0,0.0,Q1,0
2,NaN,78.0,White,HighSchool,Married,28.799999,0,0,0.0,0.0,Q3,1
3,NaN,56.0,White,Graduate,Parther,42.400002,1,0,0.0,0.0,Q3,0
4,NaN,42.0,Black,College,Divorced,20.299999,1,0,0.0,0.0,Q4,0
...,...,...,...,...,...,...,...,...,...,...,...,...
4185,Male,25.0,Other,Graduate,Never,21.000000,0,0,0.0,0.0,Q1,0
4186,Female,76.0,White,HighSchool,Widowed,21.500000,1,0,0.0,0.0,Q1,0
4187,Female,80.0,White,College,Widowed,31.000000,0,0,0.0,0.0,Q1,0
4188,Male,35.0,White,9th,Married,26.000000,0,1,0.0,0.0,Q1,0


In [46]:
df_transformed.loc[[10,20,30,40,5], 'age']

10    100.0
20    100.0
30    100.0
40    100.0
5     100.0
Name: age, dtype: float32

In [71]:
df_inverse = pm.inverse_transform(df_transformed)

In [72]:
df_inverse

,gen,age,race,edu,mar,bmi,dep,pir,gh,mets,qm,dia
0,Male,62.0,White,Graduate,Married,27.799999,0,0,0.0,0.0,Q2,1
1,Male,56.0,Black,HighSchool,Divorced,33.599998,0,0,0.0,0.0,Q3,1
2,Male,46.0,White,Graduate,Parther,27.600000,0,0,0.0,0.0,Q3,0
3,Male,45.0,Other,11th,Never,24.100000,0,0,0.0,0.0,Q3,0
4,Female,30.0,Hispanic,College,Parther,26.600000,0,0,0.0,0.0,Q4,0
...,...,...,...,...,...,...,...,...,...,...,...,...
4176,Male,25.0,Other,Graduate,Never,21.000000,0,0,0.0,0.0,Q1,0
4177,Female,76.0,White,HighSchool,Widowed,21.500000,1,0,0.0,0.0,Q1,0
4178,Female,80.0,White,College,Widowed,31.000000,0,0,0.0,0.0,Q1,0
4179,Male,35.0,White,9th,Married,26.000000,0,1,0.0,0.0,Q1,0


In [60]:
try:
    print(f'Inverse successful:\n{(df_inverse == df).all()}')
    print(f'Numeric close (age): {np.isclose(df_inverse.age, df.age).all()}')
    print(f'Numeric close (bmi): {np.isclose(df_inverse.bmi, df.bmi).all()}')
except:
    print('Not applicable.')

Not applicable.
